In [8]:
from sklearn.linear_model import LogisticRegression
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

In [2]:
import seaborn as sns

/opt/conda/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [3]:
sns.set(style='ticks', font='Noto Sans CJK JP')

In [5]:
df = sns.load_dataset('titanic')

In [6]:
df.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


In [11]:
from sklearn.datasets import load_breast_cancer

In [38]:
x, y = load_breast_cancer(return_X_y=True)

In [39]:
y.sum() / len(y)

0.6274165202108963

In [40]:
y.shape

(569,)

不均衡データの取扱をやりたいので適当に減らす

In [41]:
zero_indexes = np.where(y == 0)[0]
use_index = np.random.choice(zero_indexes, size=int(len(zero_indexes) * .2))

ones = y[np.where(y == 1)]
zeros = y[use_index]

x_ones = x[np.where(y == 1)]
x_zeros = x[use_index]

x = np.concatenate([x_ones, x_zeros])
y = np.concatenate([ones, zeros])

In [42]:
y.sum() / len(y)

0.8947368421052632

In [47]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y)

In [51]:
from sklearn.metrics import accuracy_score, roc_auc_score, f1_score, log_loss

In [52]:
def binary_metrics(y_true, predict_probability, threshold=.5):
    """
    二値分類でよく使う評価指標全部入りの dataframe を作成するメソッド
    Args:
        y_true(np.ndarray): 正解ラベルの配列. shape = (n_samples, )
        predict_probability(np.ndarray): 予測値の確率が入った配列. shape = (n_samples, )
        threshold(float): この値を超えたものをラベル 1 を予測したとみなす

    Returns:
        pd.DataFrame

    """
    auc = roc_auc_score(y_true, predict_probability)
    loss = log_loss(y_true, predict_probability)
    predict_label = np.where(predict_probability > threshold, 1, 0)
    f1 = f1_score(y_true, predict_label)
    acc = accuracy_score(y_true, predict_label)
    df_metrics = pd.DataFrame([auc, loss, f1, acc], index=['auc', 'log_loss', 'f1_score', 'accuracy'],
                              columns=['score'])
    return df_metrics

## ノーマルなアプローチ

普通に logistic 回帰をやってみる

In [59]:
clf = LogisticRegression(solver='lbfgs', max_iter=10000)
clf.fit(x_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=10000, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='lbfgs',
          tol=0.0001, verbose=0, warm_start=False)

In [60]:
pred_prob = clf.predict_proba(x_test)[:, 1]

df_metric_normal = binary_metrics(y_test, pred_prob)

In [61]:
df_metric_normal

,score
auc,1.000000
log_loss,0.041142
f1_score,0.994286
accuracy,0.990000


In [62]:
clf_balanced = LogisticRegression(class_weight='balanced', solver='lbfgs', max_iter=10000)
clf_balanced.fit(x_train, y_train)

LogisticRegression(C=1.0, class_weight='balanced', dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=10000,
          multi_class='warn', n_jobs=None, penalty='l2', random_state=None,
          solver='lbfgs', tol=0.0001, verbose=0, warm_start=False)

In [63]:
df_metric_balanced = binary_metrics(y_test, clf_balanced.predict_proba(x_test)[:, 1])

In [64]:
df_metric_balanced

,score
auc,0.998106
log_loss,0.208110
f1_score,0.958580
accuracy,0.930000
